<a href="https://colab.research.google.com/github/dharavathramdas101/Kaggle-Competitions/blob/main/Digit_Recognizer_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digit_Recognizer_kaggle

## Import tensorflow and checking version

In [1]:
#Import tensorflow and checking the versions
import tensorflow as tf
print(tf.__version__)

2.12.0


## install dependencies of kaggle

In [2]:
#Install dependencies
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pwd

/content


In [4]:
#Getting Kaggle.JSON file from Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!mkdir -p ~/.kaggle
!cp '/content/drive/My Drive/kaggle.json' ~/.kaggle/
#Changing permission
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c digit-recognizer

 65% 10.0M/15.3M [00:00<00:00, 96.5MB/s]
100% 15.3M/15.3M [00:00<00:00, 120MB/s] 


In [7]:
!unzip 'digit-recognizer.zip'

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [8]:
#!unzip 'train.csv.zip'
#!unzip 'test.csv.zip'

In [9]:
!rm -rf test.csv.zip
!rm -rf train.csv.zip

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "../content"]).decode("utf8"))

digit-recognizer.zip
drive
sample_data
sample_submission.csv
test.csv
train.csv



In [11]:
# Read training and test data files
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [19]:
train_y = train['label'].astype('float32')
train_x = train.drop(['label'],axis = 1).astype('int32')
x_test = test.astype('float32')
train_x.shape, train_y.shape, x_test.shape

((42000, 784), (42000,), (28000, 784))

In [20]:
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x / 255.0
x_test = x_test.values.reshape(-1,28,28,1)
x_test = x_test / 255.0
train_x.shape, x_test.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

## One Hot Encoding

In [21]:
#One Hot Encoding
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

## Printing original labels of to 5 rows

In [22]:
#Printing original labels of top 5 rows
print(train['label'].head())

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64


## One hot encoding of the same labels

In [23]:
#One hot encoding of the same labels
print(train_y[0:5,:])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## Defining the model

In [24]:
#Defining the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape=(28,28,1)),
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  #tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                    

## Defining the callback function to stop our training once the acceptable accuracy is reached

In [25]:
#Defining the callback function to stop our training once the acceptable accuracy is reached
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') > 0.999):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True
    
callbacks = myCallback()

## Comiling and model training with batch size = 50, epochs = 20, and optimizer = adam

In [26]:
#Compiling and model training with batch size = 50, epochs = 20, and optimizer = adam
Optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.0005, 
            beta_1=0.9, 
            beta_2=0.999, 
            epsilon=1e-07,
            name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 50, epochs = 20, callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 23s 8ms/step - loss: 0.3251 - accuracy: 0.8920
Epoch 2/20
840/840 [==============================] - 7s 8ms/step - loss: 0.0806 - accuracy: 0.9765
Epoch 3/20
840/840 [==============================] - 5s 6ms/step - loss: 0.0545 - accuracy: 0.9844
Epoch 4/20
840/840 [==============================] - 8s 9ms/step - loss: 0.0464 - accuracy: 0.9858
Epoch 5/20
840/840 [==============================] - 6s 7ms/step - loss: 0.0385 - accuracy: 0.9886
Epoch 6/20
840/840 [==============================] - 6s 7ms/step - loss: 0.0353 - accuracy: 0.9903
Epoch 7/20
840/840 [==============================] - 5s 6ms/step - loss: 0.0294 - accuracy: 0.9911
Epoch 8/20
840/840 [==============================] - 5s 6ms/step - loss: 0.0284 - accuracy: 0.9916
Epoch 9/20
840/840 [==============================] - 6s 7ms/step - loss: 0.0249 - accuracy: 0.9927
Epoch 10/20
840/840 [==============================] - 5s 6ms/step - loss: 0.0218 - accuracy: 0.993

In [28]:
results = model.predict(x_test)

# select the index with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

875/875 [==============================] - 2s 2ms/step


In [29]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)

## Submitting the results on Kagggle

In [31]:
#Submitting the results on Kaggle
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"

100% 208k/208k [00:01<00:00, 132kB/s]
Successfully submitted to Digit Recognizer

In [32]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"

100% 208k/208k [00:01<00:00, 115kB/s]
Successfully submitted to Digit Recognizer